In [153]:
%pip install langchain langchainhub langfuse openai yfinance

Note: you may need to restart the kernel to use updated packages.


In [154]:
import os
import certifi

os.environ["SSL_CERT_FILE"] = certifi.where()
print(certifi.where())

/home/user/gen-ai/lib/python3.12/site-packages/certifi/cacert.pem


In [155]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = "6e05e4deca3a4f6683cde55cd8dff597"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://eastus.api.cognitive.microsoft.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "telcogpt"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-04-01-preview"

os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-0e9fe488-cf89-460a-ad23-2b75e68e499a"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-b5ba48a0-947b-4158-a2f9-fe0fd18eec23"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"



In [156]:
import os
import yfinance as yf
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field
from langfuse import Langfuse
from langfuse.callback import CallbackHandler
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

from langchain_core.runnables import RunnableLambda, RunnableMap

In [157]:


# 🔹 Langfuse client
lf = Langfuse(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY")
)

In [158]:
# 🔹 Output schema
class SentimentOutput(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: str = Field(description="Positive, Negative, or Neutral")
    people_names: list[str]
    places_names: list[str]
    other_companies_referred: list[str]
    related_industries: list[str]
    market_implications: str
    confidence_score: float

output_parser = PydanticOutputParser(pydantic_object=SentimentOutput)



In [159]:
# Initialize Langfuse callback
langfuse_callback = CallbackHandler()
print(os.getenv("AZURE_OPENAI_API_VERSION"))

# Initialize Azure OpenAI LLM
llm = AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    model="telcogpt",
    callbacks=[langfuse_callback]
)

print(llm.invoke("Tell me something about space"))

2024-04-01-preview


Error code: 401 - {'error': {'code': '401', 'message': 'Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.'}}


AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': 'Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.'}}

In [ ]:
import requests


def fetch_news(stock_code):
    try:
        url = f"https://query1.finance.yahoo.com/v1/finance/search?q={stock_code}&esCount=1&newsCount=5"
        headers = {
            "User-Agent": os.getenv("USER_AGENT", "Mozilla/5.0")
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        news_items = data.get("news", [])
        if not news_items:
            return "No news found."
        news_text = "\n".join([item.get("title", "") for item in news_items[:5]])
        return news_text if news_text else "No news found."
    except Exception as e:
        return f"Error fetching news: {str(e)}"

# ✅ Prompt for LLM
prompt = ChatPromptTemplate.from_template(
    """You are a financial news sentiment analyzer.
Analyze the following news about {company} ({ticker}) and output a JSON like:
{{
    "company_name": "{company}",
    "stock_code": "{ticker}",
    "newsdesc": "...",
    "sentiment": "Positive/Negative/Neutral",
    "people_names": [],
    "places_names": [],
    "other_companies_referred": [],
    "related_industries": [],
    "market_implications": "",
    "confidence_score": 0.0
}}

News:
{news_text}
"""
)

# ✅ Chain definition
chain = (
    RunnableMap({
        "company": lambda x: x["company"],
        "ticker": lambda x: x["ticker"],
        "news_text": lambda x: "\n".join(fetch_news(x["ticker"]))
    }) |
    prompt |
    llm |
    output_parser
)

In [ ]:
print(fetch_news("MSFT"))

input_data = {
        "company": "Microsoft",
        "ticker": "MSFT"
    }

print("⏳ Analyzing news sentiment...")
result = chain.invoke(input_data)
print("✅ Result:\n")
print(result)

6 charts that capture Nvidia's AI-fueled rise
Nvidia has lost its shock power to investors, for now
Could Nvidia Stock Help You Retire a Millionaire?
3 No-Brainer Cloud Computing Stocks to Buy Right Now
Warren Buffett Says Buy This S&P 500 Index Fund -- It Could Soar 156% by 2030, According to a Top Wall Street Analyst
⏳ Analyzing news sentiment...
✅ Result:

company_name='Microsoft' stock_code='MSFT' newsdesc='The provided news appears to focus primarily on other companies like Nvidia and general stock market topics, rather than Microsoft specifically. There is no mention or direct reference to Microsoft in the news text.' sentiment='Neutral' people_names=['Warren Buffett'] places_names=[] other_companies_referred=['Nvidia'] related_industries=['Cloud Computing', 'AI', 'Stock Market'] market_implications="No direct implications for Microsoft were mentioned in this news. The content largely revolves around Nvidia's performance, cloud computing stocks, and general investment advice." co

In [ ]:
import os
from openai import AzureOpenAI

endpoint = "https://eastus.api.cognitive.microsoft.com/"
model_name = "gpt-4o"
deployment = "telcogpt"

subscription_key = "6e05e4deca3a4f6683cde55cd8dff597"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

Paris is one of the most beautiful cities in the world, offering countless iconic landmarks, world-class museums, charming neighborhoods, and delicious dining options. Here's a curated list of top attractions and experiences for your trip:

---

### **Iconic Landmarks and Must-See Attractions**
1. **Eiffel Tower**  
   No trip to Paris is complete without visiting the Eiffel Tower. You can take the elevator or stairs to the observation decks for stunning views of the city.

2. **Louvre Museum**  
   Home to world-famous masterpieces like the Mona Lisa and the Venus de Milo, the Louvre is a must-visit for art lovers. The building itself is magnificent, with the iconic glass pyramid.

3. **Notre-Dame Cathedral**  
   Admire the Gothic architecture, stunning stained glass windows, and historic significance of this iconic cathedral (note: restoration is ongoing, so it may not be fully accessible).

4. **Sacré-Cœur Basilica (Montmartre)**  
   Located at the highest point in Paris, this bea